In [42]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import redis
import pickle
import threading
import matplotlib.pyplot as plt
import json
import hmac
import hashlib
from collections import deque
from cryptography.fernet import Fernet
import base64
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.hkdf import HKDF



import time


SECRET_KEY = b'my_secret_key'

# Encryption key (this should be securely shared between aggregator and nodes)
key = b'password'
hkdf = HKDF(
    algorithm=hashes.SHA256(),  # You can swap this out for hashes.MD5()
    length=32,
    salt=None,    # You may be able to remove this line but I'm unable to test
    info=None,    # You may also be able to remove this line
     backend=default_backend() )
key = base64.urlsafe_b64encode(hkdf.derive(key))
cipher_suite = Fernet(key)


class KNNNode:
    def __init__(self, node_id, local_data, local_labels, distance, tamper=False):
        self.node_id = node_id
        self.local_data = local_data
        self.original_labels = local_labels
        if tamper:
            self.local_labels = self.flip_labels(local_labels)
        else:
            self.local_labels = local_labels
        self.knn = KNeighborsClassifier(n_neighbors=5, metric=distance)
        self.knn.fit(self.local_data, self.local_labels)
        self.redis_client = redis.StrictRedis(host='localhost', port=6379, db=0, password='secret_key')
        self.subscribe_channel = f'knn_{distance}_request_channel_{self.node_id}'
        self.publish_channel = 'knn_response_channel'
        self._listen_thread = threading.Thread(target=self.listen_for_requests)
        self._listen_thread.start()

    def tamper_data (self):
        self.local_labels = self.flip_labels(self.original_labels)        
        self.knn.fit(self.local_data, self.local_labels)
    
    def de_tamper_data (self):
        self.local_labels =  self.original_labels        
        self.knn.fit(self.local_data, self.local_labels)
        
    def flip_labels(self, labels):
        flipped_labels = np.copy(labels)
        for i in range(len(labels)):
            if labels[i] == 2:
                flipped_labels[i] = 1
            elif labels[i] == 1:
                flipped_labels[i] = 2
        return flipped_labels
    
        
    def sign_message(self, message):
        message_json = json.dumps(message)
        signature = hmac.new(SECRET_KEY, message_json.encode(), hashlib.sha256).hexdigest()
        return {'message': message, 'signature': signature}

    def is_encrypted(self,data):
        try:
           cipher_suite.decrypt(data)
           return True
        except Exception:
            return False

    def listen_for_requests(self):
        pubsub = self.redis_client.pubsub()
        pubsub.subscribe(self.subscribe_channel)
        for message in pubsub.listen():
            if message['type'] == 'message':
                encrypted_data = message['data']
                print(f'Before decryption {encrypted_data}')
                if(self.is_encrypted(encrypted_data)):
                    request_data = cipher_suite.decrypt(encrypted_data)
                    
                    
                else:
                    request_data = encrypted_data
                request_data = pickle.loads(request_data)

                print(f'After decryption {request_data}')  
                sample = request_data['sample']
                request_id = request_data['request_id']
                self.process_request(sample, request_id)


    def process_request(self, sample, request_id):
        distances, indices = self.knn.kneighbors([sample], n_neighbors=3)
        predictions = [self.local_labels[i] for i in indices.tolist()]
        
        response_data = {
            'node_id': self.node_id,
            'prediction': predictions[0].tolist(),
            'distances': distances[0].tolist(),
            'request_id': request_id
        }
        
        signed_message = self.sign_message(response_data)            
        self.redis_client.publish(self.publish_channel, pickle.dumps(signed_message))



# Load Dataset

from sklearn.datasets import load_digits
mnist = load_digits()
print(f'dataset shape is {mnist.data.shape}')
X = mnist.data 
y = mnist.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Simulate multiple nodes by splitting the training data
num_nodes = 3
X_train_split = np.array_split(X_train, num_nodes)
y_train_split = np.array_split(y_train, num_nodes)


# Initialize euclidean nodes with their local data
node_1 = KNNNode(node_id=1, local_data=X_train_split[0], local_labels=y_train_split[0],distance='euclidean')
node_2 = KNNNode(node_id=2, local_data=X_train_split[1], local_labels=y_train_split[1],distance='euclidean')
node_3 = KNNNode(node_id=3, local_data=X_train_split[2], local_labels=y_train_split[2],distance='euclidean')


# Initialize manhattan nodes with their local data
node_1__m = KNNNode(node_id=1, local_data=X_train_split[0], local_labels=y_train_split[0],distance='manhattan')
node_2_m = KNNNode(node_id=2, local_data=X_train_split[1], local_labels=y_train_split[1],distance='manhattan')
node_3_m = KNNNode(node_id=3, local_data=X_train_split[2], local_labels=y_train_split[2],distance='manhattan')


# Initialize cosine nodes with their local data
node_1_c = KNNNode(node_id=1, local_data=X_train_split[0], local_labels=y_train_split[0],distance='cosine')
node_2_c = KNNNode(node_id=2, local_data=X_train_split[1], local_labels=y_train_split[1],distance='cosine')
node_3_c = KNNNode(node_id=3, local_data=X_train_split[2], local_labels=y_train_split[2],distance='cosine')

dataset shape is (1797, 64)


Before decryption b'\x80\x04\x95\x86\x02\x00\x00\x00\x00\x00\x00}\x94(\x8c\x06sample\x94]\x94(G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G@\x1c\x00\x00\x00\x00\x00\x00G@(\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G@\x10\x00\x00\x00\x00\x00\x00G@0\x00\x00\x00\x00\x00\x00G@ \x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G@(\x00\x00\x00\x00\x00\x00G@&\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G@.\x00\x00\x00\x00\x00\x00G@$\x00\x00\x00\x00\x00\x00G@ \x00\x00\x00\x00\x00\x00G@\

In [39]:
mnist


{'data': array([[ 0.,  0.,  5., ...,  0.,  0.,  0.],
        [ 0.,  0.,  0., ..., 10.,  0.,  0.],
        [ 0.,  0.,  0., ..., 16.,  9.,  0.],
        ...,
        [ 0.,  0.,  1., ...,  6.,  0.,  0.],
        [ 0.,  0.,  2., ..., 12.,  0.,  0.],
        [ 0.,  0., 10., ..., 12.,  1.,  0.]]),
 'target': array([0, 1, 2, ..., 8, 9, 8]),
 'frame': None,
 'feature_names': ['pixel_0_0',
  'pixel_0_1',
  'pixel_0_2',
  'pixel_0_3',
  'pixel_0_4',
  'pixel_0_5',
  'pixel_0_6',
  'pixel_0_7',
  'pixel_1_0',
  'pixel_1_1',
  'pixel_1_2',
  'pixel_1_3',
  'pixel_1_4',
  'pixel_1_5',
  'pixel_1_6',
  'pixel_1_7',
  'pixel_2_0',
  'pixel_2_1',
  'pixel_2_2',
  'pixel_2_3',
  'pixel_2_4',
  'pixel_2_5',
  'pixel_2_6',
  'pixel_2_7',
  'pixel_3_0',
  'pixel_3_1',
  'pixel_3_2',
  'pixel_3_3',
  'pixel_3_4',
  'pixel_3_5',
  'pixel_3_6',
  'pixel_3_7',
  'pixel_4_0',
  'pixel_4_1',
  'pixel_4_2',
  'pixel_4_3',
  'pixel_4_4',
  'pixel_4_5',
  'pixel_4_6',
  'pixel_4_7',
  'pixel_5_0',
  'pixel_5_1',
 

In [34]:
x

b'\x80\x04\x95\x86\x02\x00\x00\x00\x00\x00\x00}\x94(\x8c\x06sample\x94]\x94(G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G@\x1c\x00\x00\x00\x00\x00\x00G@(\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G@\x10\x00\x00\x00\x00\x00\x00G@0\x00\x00\x00\x00\x00\x00G@ \x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G@(\x00\x00\x00\x00\x00\x00G@&\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G@.\x00\x00\x00\x00\x00\x00G@$\x00\x00\x00\x00\x00\x00G@ \x00\x00\x00\x00\x00\x00G@\x18\x00\x00\x00\x0

In [231]:
node_3.tamper_data()

In [ ]:
node_3.de_tamper_data()

In [233]:
node_3_m.tamper_data()

In [ ]:
node_3_m.de_tamper_data()

In [235]:
node_3_c.tamper_data()

In [236]:
node_3_c.de_tamper_data()

In [264]:
len(X_train_split[0])

479

In [269]:
X_test[0]

array([ 0.,  0.,  0.,  7., 12.,  0.,  0.,  0.,  0.,  0.,  4., 16.,  8.,
        0.,  0.,  0.,  0.,  0., 12., 11.,  0.,  0.,  0.,  0.,  0.,  0.,
       15., 10.,  8.,  6.,  1.,  0.,  0.,  0., 15., 16.,  8., 10.,  8.,
        0.,  0.,  0., 14.,  7.,  0.,  0., 12.,  0.,  0.,  0.,  8., 11.,
        0.,  5., 16.,  2.,  0.,  0.,  0.,  9., 14., 14.,  5.,  0.])

In [267]:
# Get values that you need you have till index 359
X_test[200]

array([ 0.,  0., 10., 12., 12., 15.,  4.,  0.,  0.,  0., 16.,  8.,  8.,
        5.,  3.,  0.,  0.,  4., 15.,  8.,  6.,  0.,  0.,  0.,  0.,  6.,
       15., 12., 14.,  8.,  0.,  0.,  0.,  0.,  1.,  0.,  2., 16.,  0.,
        0.,  0.,  0.,  0.,  0.,  0., 14.,  3.,  0.,  0.,  0., 11.,  4.,
        8., 15.,  3.,  0.,  0.,  0., 10., 16., 15.,  5.,  0.,  0.])